In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [6]:
train_df = pd.read_csv('naver_movie_train.tsv', sep='\t')
test_df = pd.read_csv('naver_movie_test.tsv', sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


tokenizer 함수 정의

In [7]:
from konlpy.tag import Okt
okt = Okt()


In [8]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()

In [13]:
def okt_tokenizer(text):
    morphs = okt.morphs(text,stem=True)
    tokens = [word for word in morphs if word not in stopwords]
    return tokens

- TfidfVectorizer 변환

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), max_df=0.9)

In [15]:
%time tvect.fit(train_df.document)

Wall time: 3min 59s


TfidfVectorizer(max_df=0.9, ngram_range=(1, 2),
                tokenizer=<function okt_tokenizer at 0x0000017455CBE310>)

In [17]:
%time X_train_tv = tvect.transform(train_df.document)

Wall time: 4min 13s


In [18]:
%time X_test_tv = tvect.transform(test_df.document)

Wall time: 1min 16s


In [26]:
y_train = train_df.label.values 
y_test = test_df.label.values 

naive bays 분류기

In [27]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [28]:
nb.fit(X_train_tv, y_train)

MultinomialNB()

In [29]:
nb.score(X_test_tv, y_test)

0.8599653025818961

실제 테스트

In [30]:
reviews = ['혹시 0점 없냐?? 여자배우는 그나마 괜찮은데 남자배우는 도대체 어디서 데려온거냐??? 저렇게 못생긴애 찾기도 힘들겠다', '이런 한심한 영화 볼 돈이면 추운날 따뜻한 국밥 한그릇 드시길 추천합니다.']


In [32]:
reviews_tv = tvect.transform(reviews)
nb.predict(reviews_tv)

array([0, 0], dtype=int64)